In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

TOKEN = os.environ["INVEST_TOKEN"]

In [2]:
from datetime import timedelta
from pandas import DataFrame
from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now

all_candles = []

def main():

    with Client(TOKEN) as client:
        for candle in client.get_all_candles(
            figi="BBG004730N88",    # FIGI Composite of SBER
            from_=now() - timedelta(days=90),
            interval=CandleInterval.CANDLE_INTERVAL_DAY,
        ):
            all_candles.append(
                {
                    "Time": candle.time,
                    "Open": candle.open,
                    "Close": candle.close,
                    "Max": candle.high,
                    "Min": candle.low,
                    "Volume": candle.volume,
                }
            )

    return 0


if __name__ == "__main__":
    main()


In [4]:
from modules.modules import Quotation, factor
def convert_quotation(q: Quotation):
    return q.units + q.nano / factor

In [5]:
df = DataFrame(all_candles)
for column in ["Open", "Close", "Max", "Min"]:
    df[column] = df[column].apply(convert_quotation)

df.set_index("Time", inplace=True)
df

,Open,Close,Max,Min,Volume
Time,,,,,
2024-08-19 00:00:00+00:00,275.00,268.17,275.56,265.02,6862620
2024-08-20 00:00:00+00:00,268.35,266.20,269.49,263.36,5427613
2024-08-21 00:00:00+00:00,266.20,266.93,267.80,263.86,3712748
2024-08-22 00:00:00+00:00,266.93,261.60,268.50,260.15,4790753
2024-08-23 00:00:00+00:00,261.61,259.50,262.80,255.56,7671918
...,...,...,...,...,...
2024-11-11 00:00:00+00:00,257.99,260.73,261.30,256.49,6784794
2024-11-12 00:00:00+00:00,259.99,255.77,260.70,255.27,4092685
2024-11-13 00:00:00+00:00,255.29,254.70,258.96,254.05,4454002


In [6]:
from modules.modules import Window

wi = Window(small=9, medium=26, large=52)

In [7]:
df["Tenkan-sen"]=(df["Max"].rolling(window=wi.small).max() + df["Min"].rolling(window=wi.small).min()) / 2
df['Kijun-sen'] = (df['Max'].rolling(window=wi.medium).max() + df['Min'].rolling(window=wi.medium).min()) / 2
df['Senkou Span A'] = ((df['Tenkan-sen'] + df['Kijun-sen']) / 2).shift(wi.medium)
df['Senkou Span B'] = ((df['Max'].rolling(window=wi.large).max() + df['Min'].rolling(window=wi.large).min()) / 2).shift(wi.medium)
df['Chikou Span'] = df['Close'].shift(-wi.medium)

In [8]:
df

,Open,Close,Max,Min,Volume,Tenkan-sen,Kijun-sen,Senkou Span A,Senkou Span B,Chikou Span
Time,,,,,,,,,,
2024-08-19 00:00:00+00:00,275.00,268.17,275.56,265.02,6862620,NaN,NaN,NaN,NaN,272.92
2024-08-20 00:00:00+00:00,268.35,266.20,269.49,263.36,5427613,NaN,NaN,NaN,NaN,267.80
2024-08-21 00:00:00+00:00,266.20,266.93,267.80,263.86,3712748,NaN,NaN,NaN,NaN,267.88
2024-08-22 00:00:00+00:00,266.93,261.60,268.50,260.15,4790753,NaN,NaN,NaN,NaN,268.07
2024-08-23 00:00:00+00:00,261.61,259.50,262.80,255.56,7671918,NaN,NaN,NaN,NaN,268.49
...,...,...,...,...,...,...,...,...,...,...
2024-11-11 00:00:00+00:00,257.99,260.73,261.30,256.49,6784794,247.935,250.785,261.2725,NaN,NaN
2024-11-12 00:00:00+00:00,259.99,255.77,260.70,255.27,4092685,247.935,249.645,261.2375,NaN,NaN
2024-11-13 00:00:00+00:00,255.29,254.70,258.96,254.05,4454002,247.935,249.645,260.8700,NaN,NaN


In [9]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['Max'],
                low=df['Min'],
                close=df['Close'],
                increasing_line_color='black',
                decreasing_line_color='grey')])

fig.add_trace(go.Scatter(x=df.index, y=df['Tenkan-sen'], mode='lines', name='Tenkan-sen', line=dict(color='red')))
fig.add_trace(go.Scatter(x=df.index, y=df['Kijun-sen'], mode='lines', name='Kijun-sen', line=dict(color='blue')))

fig.add_trace(go.Scatter(x=df.index, y=df['Senkou Span A'], mode='lines', name='Senkou Span A', line=dict(color='yellow')))
fig.add_trace(go.Scatter(x=df.index, y=df['Senkou Span B'], mode='lines', name='Senkou Span B', line=dict(color='purple')))

fig.add_trace(go.Scatter(x=df.index, y=df['Senkou Span A'], fill='tonexty', fillcolor='rgba(0,100,80,0.2)', line=dict(color='rgba(255,257,255,0)'), name='Kumo'))

fig.add_trace(go.Scatter(x=df.index, y=df['Chikou Span'], mode='lines', name='Chikou Span', line=dict(color='green')))

buySignals = (df['Tenkan-sen'] > df['Kijun-sen']) & (df['Tenkan-sen'].shift(1) <= df['Kijun-sen'].shift(1))
sellSignals = (df['Tenkan-sen'] < df['Kijun-sen']) & (df['Tenkan-sen'].shift(1) >= df['Kijun-sen'].shift(1))

fig.add_trace(go.Scatter(x=df.index[buySignals], y=df['Min'][buySignals], mode='markers', marker=dict(color='green', size=8), name='Buy Signal'))
fig.add_trace(go.Scatter(x=df.index[sellSignals], y=df['Max'][sellSignals], mode='markers', marker=dict(color='red', size=8), name='Sell Signal'))

fig.show()